### Step 1: Import Libraries and Set up Snowflake Connections

In [72]:
import snowflake.connector
import pandas as pd
## Store the credentials in a separate file called config.py and include that file in the .gitignore.
## Make sure the credentials are not exposed online when publishing to GitHub.
import config  

In [3]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=config.SNOWFLAKE_USER,
    password=config.SNOWFLAKE_PASSWORD,
    account=config.SNOWFLAKE_ACCOUNT,
    warehouse=config.SNOWFLAKE_WAREHOUSE,
    database=config.SNOWFLAKE_DATABASE,
    schema=config.SNOWFLAKE_SCHEMA,
    role=config.SNOWFLAKE_ROLE
)

# Create a cursor to execute SQL commands
cursor = conn.cursor()

### Step 2: Import data from Snowflake

In [85]:
# Query the data and load it into a DataFrame
users_query = "SELECT * FROM stg_users"
df_users = pd.read_sql(users_query, conn)

receipts_query = "SELECT * FROM stg_receipts"
df_receipts = pd.read_sql(receipts_query, conn)

brands_query = "SELECT * FROM stg_brands"
df_brands = pd.read_sql(brands_query, conn)

receipts_items_query = "SELECT * FROM stg_receipts_items"
df_receitps_items = pd.read_sql(receipts_items_query, conn)


/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/ipykernel_48943/2188856631.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_users = pd.read_sql(users_query, conn)
/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/ipykernel_48943/2188856631.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_receipts = pd.read_sql(receipts_query, conn)
/var/folders/0q/_0939hcn7tb0ch1qcjyxjsxw0000gn/T/ipykernel_48943/2188856631.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_brands = pd.read_sql(brands_query, conn)
/var/folder

### Step 3: Create reusable quality check functions

In [102]:
# Define the function to check unique values
def check_unique(df, column_list):
    result_df = pd.DataFrame(columns=['Column','Total Counts','Unique Counts','Unique%'])
    total_records = len(df)
    for col in column_list:
        unique_records = df[col].nunique()
        per = round(unique_records / total_records * 100,2)
        result_df.loc[len(result_df)]=[col,total_records,unique_records, per]
    return result_df

In [103]:
# Define the function to check null values
def check_null(df, column_list):
    result_df = pd.DataFrame(columns=['Column','Total Counts','Null Counts','Null%'])
    total_records = len(df)
    for col in column_list:
        null_count = df[col].isnull().sum()
        per = round(null_count / total_records * 100,2)
        result_df.loc[len(result_df)]=[col,total_records,null_count, per]
#         report = f"Out of {total_records} {column_name} records, {per}% are unique."
    return result_df

### Step 4: Data Quality Check for Users Table

In [79]:
## Check the uniqueness of ID column
check_unique(df_users,['ID'])

,Column,Total Counts,Unique Counts,Unique%
0,ID,495,212,42.83


In [80]:
## Check the null records of various column
check_null(df_users, ['ID','ACTIVE','CREATEDDATE','LASTLOGIN','ROLE','SIGNUPSOURCE','STATE'])

,Column,Total Counts,Null Counts,Null%
0,ID,495,0,0.00
1,ACTIVE,495,0,0.00
2,CREATEDDATE,495,0,0.00
3,LASTLOGIN,495,62,12.53
4,ROLE,495,0,0.00
5,SIGNUPSOURCE,495,48,9.70
6,STATE,495,56,11.31


#### Data Issues Summary for Users Table 

- **Duplicated Users ID**: 
The User ID field is intended to serve as the primary key for the table and should be unique for each record. However, only 43% of the records have unique IDs, indicating that more than half of the records are duplicates. Further investigation is required to determine whether these duplicate IDs are errors or if they need to be de-duplicated.
- **Incomplete Users Attributes**: Some columns, such as last login date, signup source, and state, contain null values. The absence of these values may not necessarily indicate a data issue, as it could be related to business logic. For example, the “state” field may not be required during user sign-up. It is important to further investigate the reasons behind the missing values in these attributes.

### Step 5: Data Quality Check for Brands Table

In [81]:
## Check the uniqueness of various columns
check_null(df_brands,['ID','BARCODE','BRANDCODE','CATEGORY','CATEGORYCODE','CPG_ID','CPG_ID','NAME','TOPBRAND'])

,Column,Total Counts,Null Counts,Null%
0,ID,1167,0,0.00
1,BARCODE,1167,0,0.00
2,BRANDCODE,1167,234,20.05
3,CATEGORY,1167,155,13.28
4,CATEGORYCODE,1167,650,55.70
5,CPG_ID,1167,0,0.00
6,CPG_ID,1167,0,0.00
7,NAME,1167,0,0.00
8,TOPBRAND,1167,612,52.44


In [82]:
## Check the null records of various column
check_unique(df_brands,['ID','BARCODE','BRANDCODE','NAME'])

,Column,Total Counts,Unique Counts,Unique%
0,ID,1167,1167,100.00
1,BARCODE,1167,1160,99.40
2,BRANDCODE,1167,897,76.86
3,NAME,1167,1156,99.06


In [109]:
## Brandcodes in receipts_items Table Not Present in brands Table
missing_brandcodes = df_receitps_items[~df_receitps_items['BRANDCODE'].isin(df_brands['BRANDCODE'])]
missing_brandcodes['BRANDCODE'].nunique()

186


#### Data Issues Summary for Brands Table 

- **Lack of Uniqueness in BARCODE and BRANDCODE**: Each row in the brands table is meant to represent a unique brand, and therefore, both the barcode and brandcode should be unique. However, neither the barcode nor brandcode are unique, as multiple brand IDs share the same barcode and brandcode. This creates challenges when trying to join the brands table with the receipts table.
- **Null values on many important columns**: Many columns, such as brandcode, category, categorycode, and topbrand, contain a significant number of null values.
- **Missing Brandcode in the brands tables**: About 186 brands appear on scanned receipts but do not exist in the brands table. This discrepancy highlights a significant issue in our dimension table, potentially impacting data integrity and reporting accuracy.

### Step 5: Data Quality Check for Receipts Table

In [83]:
check_unique(df_receipts,['ID'])

,Column,Total Counts,Unique Counts,Unique%
0,ID,1119,1119,100.0


In [110]:
check_null(df_receipts,['ID','BONUSPOINTSEARNED','FINISHEDDATE',\
                        'POINTSAWARDEDDATE','POINTSEARNED','PURCHASEDATE','TOTALSPENT'])

,Column,Total Counts,Null Counts,Null%
0,ID,1119,0,0.00
1,BONUSPOINTSEARNED,1119,575,51.39
2,FINISHEDDATE,1119,551,49.24
3,POINTSAWARDEDDATE,1119,582,52.01
4,POINTSEARNED,1119,510,45.58
5,PURCHASEDATE,1119,448,40.04
6,TOTALSPENT,1119,435,38.87


In [96]:
##Returns records where points awarded date is populated but points are null
##or points awarded date is null but points are not null.
filtered_df1 = df_receipts[((df_receipts['POINTSAWARDEDDATE'].notnull()) & (df_receipts['POINTSEARNED'].isnull())) |
                 ((df_receipts['POINTSAWARDEDDATE'].isnull()) & (df_receipts['POINTSEARNED'].notnull()))]

filtered_df1[['POINTSAWARDEDDATE','POINTSEARNED']][:5]

,POINTSAWARDEDDATE,POINTSEARNED
2,NaT,5.0
12,NaT,8850.0
62,NaT,750.0
123,NaT,0.0
129,NaT,25.0


In [97]:
##Returns records where PURCHASEDITEMCOUNT is populated but REWARDSRECEIPTITEMLIST are null
##or PURCHASEDITEMCOUNT is null but REWARDSRECEIPTITEMLIST are not null.

filtered_df2 = df_receipts[((df_receipts['PURCHASEDITEMCOUNT'].notnull()) & \
                            (df_receipts['REWARDSRECEIPTITEMLIST'].isnull())) |
                 ((df_receipts['PURCHASEDITEMCOUNT'].isnull()) & \
                  (df_receipts['REWARDSRECEIPTITEMLIST'].notnull()))]

filtered_df2[['PURCHASEDITEMCOUNT','REWARDSRECEIPTITEMLIST']][:5]

,PURCHASEDITEMCOUNT,REWARDSRECEIPTITEMLIST
175,0.0,None
210,NaN,"[\n {\n ""description"": ""flipbelt level ter..."
211,NaN,"[\n {\n ""description"": ""flipbelt level ter..."
213,NaN,"[\n {\n ""description"": ""flipbelt level ter..."
214,NaN,"[\n {\n ""description"": ""flipbelt level ter..."


In [106]:
##Returns records where REWARDSRECEIPTSTATUS is REJECTED but POINTSAWARDEDDATE is populated
filtered_df3 = df_receipts[((df_receipts['REWARDSRECEIPTSTATUS']=='REJECTED') & \
                            (df_receipts['POINTSAWARDEDDATE'].notnull()))]

filtered_df3[['REWARDSRECEIPTSTATUS','POINTSAWARDEDDATE']][:5]

,REWARDSRECEIPTSTATUS,POINTSAWARDEDDATE
13,REJECTED,2020-10-30 20:18:00
517,REJECTED,2020-11-09 05:36:52
682,REJECTED,2020-11-15 23:44:17
698,REJECTED,2020-11-16 20:00:24


#### Data Issues Summary for Receipts Table 
- **Null values on PURCHASEDATE & TOTALSPEND**: The null check summary table shows many columns with high null counts. However, not all nulls indicate data issues. For instance, BONUSPOINTSEARNED might be null if no bonus points were awarded, and FINISHEDDATE or POINTSAWARDEDDATE could be null if the receipt isn’t finalized or points haven’t been awarded yet. However, nulls in PURCHASEDATE and TOTALSPENT may signal data issues, as every receipt should have a purchase date, and the total amount should be zero or greater—not null.<br><br>

- **Data Inconsistency within the table**:
    - **Points_Awarded_Date & Points_Earned:**<br> Many records show points awarded date but have points earned as null, or vice versa. Ideally, both columns should either be filled together or both be null, as they both reflect the fact that points have been awarded.. This inconsistency suggests a data issue.
    - **Purchased_Item_Counts & Rewards_Receipts_Item_List:**<br> Many records have a null value for the purchased item count but a non-null item list, or vice versa, which is contradictory. The purchased item count should align with the receipt item list.
    - **Points_Rewarded_Date populated for rejected Receipts:**<br> Many records have a points rewards date filled in even though the status is rejected, suggesting that points were rewarded for rejected receipts, which is contradictory.